# Econ 8210 Quant Macro, Homework 1
## Part 3 - Dynamic Programming
Haosi Shen, Fall 2024

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import math
import time
import torch

np.random.seed(42) 

Consider the following model. There is a representative household with preferences over private consumption $c_t$, government consumption $g_t$, and labor $l_t$:

\begin{equation*}
\mathbb{E}_0 \sum_{t=0}^{\infty} 0.97^t \Big( \log{c_t}+0.2\log{g_t} -\frac{l_t^2}{2} \Big)
\end{equation*}

The household consumes, saves, works, and pays labor taxes, with a budget constraint:

\begin{equation*}
c_t + i_t = (1-\tau_t)w_t l_t + r_t k_t
\end{equation*}

where the tax rate follows a 3-state Markov chain that takes values in:
\begin{equation*}
\tau_t \in \{0.20, 0.25, 0.30\}
\end{equation*}


There is a production function:
\begin{equation*}
c_t + i_t + g_t = e^{z_t} k_{t}^{0.33} l_{t}^{0.67}
\end{equation*}

with a law of motion for capital with investment adjustment costs:
\begin{equation*}
k_{t+1} = 0.9 k_t + \Big(1-0.05 \big(\frac{i_t}{i_{t-1}}-1\big)^2 \Big) i_t
\end{equation*}

and a technology level $z_t$ that follows a 5-state Markov chain that takes values in:
\begin{equation*}
z_t \in \{−0.0673, −0.0336, 0, 0.0336, 0.0673\}
\end{equation*}

Finally, there is a government that uses taxes to pay for government consumption with a
balanced budget period by period:
\begin{equation*}
g_t = \tau_t w_t l_t
\end{equation*}

### Social Planner's Problem

The social planner solves

\begin{equation*}
\max_{\{c_t, l_t, i_t\}} \mathbb{E}_0 \left[ \sum_{t=0}^{\infty} \beta^t \Big( \log{c_t}+0.2\log{g_t} -\frac{l_t^2}{2} \Big) \right]
\end{equation*}
subject to:
\begin{align*}
c_t + i_t + \tau_t w_t l_t &= e^{z_t} k_{t}^{\alpha} l_{t}^{1-\alpha} \\
k_{t+1} &= (1-\delta) k_t + \Big(1-0.05 \big(\frac{i_t}{i_{t-1}}-1\big)^2 \Big) i_t
\end{align*}
where $\beta = 0.97,\; \alpha=0.33,\;\delta=0.1 $.

**Recursive Formulation**
> *State Variables*: $\; k_t,\; i_{t-1},\; \tau_t,\; z_t$ \
> *Control Variables*: $\; c_t,\; i_t,\; l_t$

**Bellman Equation**

\begin{equation*}
V(k,i_{-1},\tau,z) = \max_{c,l,i}\; \left\{ U(c,l) + \beta \sum_{\tau', z'} \pi(\tau', z' | \tau, z) V(k',i,\tau',z') \right\}
\end{equation*}

where:
\begin{align*}
k' &= (1-\delta) k + \Big(1-0.05 \big(\frac{i}{i_{-1}}-1\big)^2 \Big) i \\
U(c,l) &= \log{c} + 0.2 \log{\big(\tau(1-\alpha)e^z k^\alpha l^{1-\alpha} \big)}-\frac{l^2}{2} \\
c &= \big(1-\tau(1-\alpha)\big) e^z k^\alpha l^{1-\alpha} -i
\end{align*}

**Revised Bellman Equation**

\begin{align*}
V(k,i_{-1},\tau,z) = \max_{l,i}\; & \Bigg\{ \log{\bigg(\big(1-\tau(1-\alpha)\big) e^z k^\alpha l^{1-\alpha} -i\bigg)} + 0.2 \log{\big(\tau(1-\alpha)e^z k^\alpha l^{1-\alpha} \big)}-\frac{l^2}{2} \\
& + \beta \sum_{\tau', z'} \pi(\tau', z' | \tau, z)\; V\bigg((1-\delta) k + \Big(1-0.05 \big(\frac{i}{i_{-1}}-1\big)^2 \Big) i,i,\tau',z'\bigg) \Bigg\}
\end{align*}


### Deterministic Steady State

To solve for $c_{ss}, l_{ss}, i_{ss}, k_{ss}$, we solve the following system of equations

* Production Function
$$ y_{ss} = k_{ss}^{\alpha} l_{ss}^{1-\alpha} $$    
* Resource Constraint
$$ c_{ss}+0.1 k_{ss} + 0.25 w_{ss}l_{ss} = k_{ss}^{\alpha} l_{ss}^{1-\alpha} $$
* Steady State Investment
$$ i_{ss} = 0.1 k_{ss} $$
* Steady State Wage
$$ w_{ss}=(1-\alpha)\Big(\frac{k_{ss}}{l_{ss}}\Big)^{\alpha} $$
* Optimal Labor
$$  $$

In [2]:
# ===================== DEFINE PARAMETERS =====================
beta = 0.97  # time discount factor
alpha = 0.33  # capital share
delta = 0.1  # capital depreciation rate
theta_c = 0.2  # weight on govt consumption in util
k_ss = 1.0  # steady-state capital (init guess)
i_ss = 0.1  # steady-state investment (init guess)

# Tax rates and transition matrix
tau_values = np.array([0.2, 0.25, 0.3])
tau_trans = np.array([[0.9, 0.1, 0.0],
                      [0.05, 0.9, 0.05],
                      [0.0, 0.1, 0.9]])

# Technology levels and transition matrix
z_values = np.array([-0.0673, -0.0336, 0, 0.0336, 0.0673])
z_trans = np.array([[0.9727, 0.0273, 0, 0, 0],
                    [0.0041, 0.9806, 0.0153, 0, 0],
                    [0, 0.0082, 0.9836, 0.0082, 0],
                    [0, 0, 0.0153, 0.9806, 0.0041],
                    [0, 0, 0, 0.0273, 0.9727]])


### Value Function Iteration with Fixed Grid


In [2]:
'''
# Grids for capital and lagged investment
num_k = 250  # number of grid points for capital
num_i = 50   # number of grid points for investment

k_grid = np.linspace(0.7 * k_ss, 1.3 * k_ss, num_k)  # ±30% around k_ss
i_grid = np.linspace(0.5 * i_ss, 1.5 * i_ss, num_i)  # ±50% around i_ss

# Initialize value function
V = np.zeros((num_k, num_i, len(tau_values), len(z_values)))

# Helper function to calculate the next capital stock
def next_capital(k, i, i_lag):
    return 0.9 * k + (1 - 0.05 * ((i / i_lag) - 1) ** 2) * i

# Bellman iteration
tol = 1e-6  # convergence tolerance
max_iter = 1000  # max iterations

for iter in range(max_iter):
    V_new = np.copy(V)
    diff = 0

    # Loop over all state variables
    for k_idx, k in enumerate(k_grid):
        for i_idx, i_lag in enumerate(i_grid):
            for tau_idx, tau in enumerate(tau_values):
                for z_idx, z in enumerate(z_values):
                    
                    max_val = -np.inf
                    best_c, best_l, best_i = None, None, None

                    # Loop over control variables: consumption, labor, investment
                    for i in i_grid:
                        # Compute the implied consumption and labor
                        l = (k ** alpha * np.exp(z)) ** (1/0.67)
                        c = (1 - tau) * l + k * alpha * np.exp(z) - i

                        # Ensure feasibility
                        if c > 0 and l > 0:
                            # Government consumption
                            g = tau * l

                            # Compute utility
                            utility = np.log(c) + theta_c * np.log(g) - 0.5 * l ** 2

                            # Compute next state and expected value
                            k_next = next_capital(k, i, i_lag)

                            # Interpolate value function for next period
                            v_next = 0
                            for tau_next_idx, tau_next in enumerate(tau_values):
                                for z_next_idx, z_next in enumerate(z_values):
                                    prob_tau = tau_trans[tau_idx, tau_next_idx]
                                    prob_z = z_trans[z_idx, z_next_idx]
                                    v_next += prob_tau * prob_z * V_new[k_idx, i_idx, tau_next_idx, z_next_idx]

                            # Calculate total value
                            total_val = utility + beta * v_next

                            # Update value function if higher value found
                            if total_val > max_val:
                                max_val = total_val
                                best_c, best_l, best_i = c, l, i

                    # Update value function
                    V_new[k_idx, i_idx, tau_idx, z_idx] = max_val
                    diff = max(diff, abs(V_new[k_idx, i_idx, tau_idx, z_idx] - V[k_idx, i_idx, tau_idx, z_idx]))

    # Check for convergence
    if diff < tol:
        print(f'Converged in {iter + 1} iterations')
        break

    # Update the value function
    V = V_new

print('Value Function Iteration complete.')

'''

"\n# Grids for capital and lagged investment\nnum_k = 250  # number of grid points for capital\nnum_i = 50   # number of grid points for investment\n\nk_grid = np.linspace(0.7 * k_ss, 1.3 * k_ss, num_k)  # ±30% around k_ss\ni_grid = np.linspace(0.5 * i_ss, 1.5 * i_ss, num_i)  # ±50% around i_ss\n\n# Initialize value function\nV = np.zeros((num_k, num_i, len(tau_values), len(z_values)))\n\n# Helper function to calculate the next capital stock\ndef next_capital(k, i, i_lag):\n    return 0.9 * k + (1 - 0.05 * ((i / i_lag) - 1) ** 2) * i\n\n# Bellman iteration\ntol = 1e-6  # convergence tolerance\nmax_iter = 1000  # max iterations\n\nfor iter in range(max_iter):\n    V_new = np.copy(V)\n    diff = 0\n\n    # Loop over all state variables\n    for k_idx, k in enumerate(k_grid):\n        for i_idx, i_lag in enumerate(i_grid):\n            for tau_idx, tau in enumerate(tau_values):\n                for z_idx, z in enumerate(z_values):\n                    \n                    max_val = -np.i

### Value Function Iteration with Endogenous Grid